# Getting OCR-ed books from gallica.bnf.fr

The whole digital collection for various centuries is at https://gallica.bnf.fr/html/und/litteratures/les-classiques-de-la-litterature-acces-par-periode?mode=desktop. Our focus is on the following collections:

1. https://gallica.bnf.fr/html/und/litteratures/les-classiques-de-la-litterature-du-xviiie-siecle?mode=desktop
2. https://gallica.bnf.fr/html/und/litteratures/les-classiques-de-la-litterature-du-xixe-siecle?mode=desktop
3. https://gallica.bnf.fr/html/und/litteratures/les-classiques-de-la-litterature-du-xxe-siecle?mode=desktop

Our task is to download all OCR-ed books from that collection for three centuries. The problem is that the APIs (https://api.bnf.fr/api-gallica-de-recherche#/recherche and https://api.bnf.fr/fr/api-document-de-gallica#/texte%20brut/get__ark__texteBrut) do not provide search over collections.

## Get all authors from the collection and the links to their pages

In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import urllib.request


urls = ['https://gallica.bnf.fr/html/und/litteratures/les-classiques-de-la-litterature-du-xviiie-siecle?mode=desktop',
        'https://gallica.bnf.fr/html/und/litteratures/les-classiques-de-la-litterature-du-xixe-siecle?mode=desktop',
        'https://gallica.bnf.fr/html/und/litteratures/les-classiques-de-la-litterature-du-xxe-siecle?mode=desktop']
authors = []
for url in urls:
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')

    for link in soup.find_all("div", {"class": "titre"}):
        try:
            authors.append( [link.a.get('href'), link.a.get('title')] )
        except Exception:
            pass
len(authors)

## Get all links to all books of all authors, titles of books, contributors and license

In [ ]:
def request_gallica(query, keywords, filters, startRecord=1):
    url='https://gallica.bnf.fr/SRU'
    params = {'operation': 'searchRetrieve',
              'exactSearch': False,
              'version': '1.2',
              'query': query,
              'collapsing': True,
              'keywords': keywords,
              'startRecord': startRecord,
              'maximumRecords': 50,
              'filter': filters,
              }
    try:
        r = requests.get(url, params)
        r.raise_for_status()
    except Exception: # due to error 500 sometimes
        r = requests.get(url, params)
        r.raise_for_status()
    return BeautifulSoup(r.text, 'lxml')

def merge_pages_of_records(soup, link):
    records = []
    for record in soup.find_all("srw:record"):
        identifier = record.find('dc:identifier').text
        title = record.find('dc:title').text
        contributors = [s.text for s in record.find_all('dc:contributor')]
        rights = [s.text for s in record.find_all('dc:rights')]
        records.append([link, identifier, title, contributors, rights])
    return records

records = []
errors = []
for i, (link, title) in tqdm(enumerate(authors)):
    if link.startswith('https://gallica.bnf.fr/ark:'):
        records.append([link, link, title, [], []])
    else:
        parts_of_link = [str(s) for s in link.split('&')]
        query = [s.replace('query=', '') for s in parts_of_link if s.startswith('query')]
        filters = [s.replace('filter=', '') for s in parts_of_link if s.startswith('filter')]
        keywords = [s.replace('keywords=', '') for s in parts_of_link if s.startswith('keywords')]
        try:
            soup = request_gallica(query, keywords, filters)
            number_of_records = int(soup.find("srw:numberofrecords").text)
            records.extend( merge_pages_of_records(soup, link) )
            pages = number_of_records // 50
            if pages>0 and number_of_records!=50:
                sRecord = 51
                for page in range(1,pages+1):
                    sRecord += 50 
                    soup = request_gallica(query, keywords, filters, startRecord=sRecord)
                    records.extend(merge_pages_of_records(soup, link))
        except Exception:
            errors.append([i, link, title, sRecord])

## Saving metadata and problematic cases

In [ ]:
print( len(errors), len(records) )

In [ ]:
num = 0
for rec in records:
    if rec[1].startswith('https://gallica.bnf.fr/ark'):
        num += 1
num

In [ ]:
import csv
with open("records.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(['identifier', 'title', 'contributors', 'rights']) #'link', 
    writer.writerows(records)

with open("errors.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(['index', 'link', 'title']) # , 'sRecord'
    writer.writerows(errors)

## Download all OCR-ed books into /data folder

In [ ]:
for rec in tqdm(records):
    [link, url, title, contributors, license] = rec
    if url.startswith('https://gallica.bnf.fr/ark'): 
        filename = url.replace('https://gallica.bnf.fr/', '').replace('/', '|')
        url = url + '.texteBrut'
        urllib.request.urlretrieve(url, '/Users/teddy/Documents/digitale_Romanistik/data/' + filename + '.html')